In [2]:
import time
import os
import sys
import pandas as pd
from pathlib import Path

sys.path.append(os.path.join(Path().resolve(), "utils"))

from TestUser import TestUser
from get_resource_status import get_resource_status


challenge_id = 1

tries_per_amount = 1

min_concurrent_instances = 1
max_concurrent_instances = 5
step_concurrent_instances = 1

duration_seconds_per_test = 10
resource_status_polling_interval_seconds = 1

timeout_until_measurement_seconds = 10


df = pd.DataFrame(columns=[
    'concurrent_instances',
    'try_number',
    'average_cpu_percent',
    'max_cpu_percent',
    'min_cpu_percent',
    'average_memory_percent',
    'max_memory_percent',
    'min_memory_percent'
])



for amount in range(min_concurrent_instances, max_concurrent_instances + 1, step_concurrent_instances):
    for try_number in range(1, tries_per_amount + 1):
        print(f'Starting test with {amount} concurrent instances, try {try_number}...')

        users = []

        for i in range(amount):
            user = TestUser(
                username=f'testuser_{i+1}',
                password='testpass',
                email=f'test{i+1}@test.com'
            )
            users.append(user)

        for user in users:
            user.create()

        for user in users:
            user.launch_challenge(challenge_id)

        print('Waiting for all challenges to be running...')
        time.sleep(timeout_until_measurement_seconds)

        start_time = time.time()

        print('Collecting resource status data...')
        cpu_percents = []
        memory_percents = []
        while True:
            status = get_resource_status()
            if status is None:
                print('Failed to get resource status, retrying...')
                time.sleep(resource_status_polling_interval_seconds)
                continue

            cpu_percents.append(status['cpu'] * 100)
            memory_percent = status['memory']['used'] / status['memory']['total'] * 100
            memory_percents.append(memory_percent)

            while time.time() - start_time < resource_status_polling_interval_seconds:
                pass

            if time.time() - start_time >= duration_seconds_per_test:
                break

        for user in users:
            user.stop_challenge(challenge_id)

        for user in users:
            user.delete()

        average_cpu_percent = sum(cpu_percents) / len(cpu_percents)
        max_cpu_percent = max(cpu_percents)
        min_cpu_percent = min(cpu_percents)

        average_memory_percent = sum(memory_percents) / len(memory_percents)
        max_memory_percent = max(memory_percents)
        min_memory_percent = min(memory_percents)

        df = pd.concat([df, pd.DataFrame([{
            'concurrent_instances': amount,
            'try_number': try_number,
            'average_cpu_percent': average_cpu_percent,
            'max_cpu_percent': max_cpu_percent,
            'min_cpu_percent': min_cpu_percent,
            'average_memory_percent': average_memory_percent,
            'max_memory_percent': max_memory_percent,
            'min_memory_percent': min_memory_percent
        }])], ignore_index=True)

        print(f'Test with {amount} concurrent instances, try {try_number} completed.')
        print(f'Average CPU Percent: {average_cpu_percent:.2f}%')
        print(f'Max CPU Percent: {max_cpu_percent:.2f}%')
        print(f'Min CPU Percent: {min_cpu_percent:.2f}%')
        print(f'Average Memory Percent: {average_memory_percent:.2f}%')
        print(f'Max Memory Percent: {max_memory_percent:.2f}%')
        print(f'Min Memory Percent: {min_memory_percent:.2f}%')
        print()

Starting test with 1 concurrent instances, try 1...
Waiting for all challenges to be running...


/tmp/ipykernel_42105/4085229176.py:99: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([{


Test with 1 concurrent instances, try 1 completed.
Average CPU Percent: 6.71%
Max CPU Percent: 9.01%
Min CPU Percent: 0.00%
Average Memory Percent: 1.86%
Max Memory Percent: 1.91%
Min Memory Percent: 1.83%

Starting test with 2 concurrent instances, try 1...
Waiting for all challenges to be running...
Test with 2 concurrent instances, try 1 completed.
Average CPU Percent: 12.41%
Max CPU Percent: 15.30%
Min CPU Percent: 0.00%
Average Memory Percent: 2.15%
Max Memory Percent: 2.26%
Min Memory Percent: 2.07%

Starting test with 3 concurrent instances, try 1...
Waiting for all challenges to be running...
Test with 3 concurrent instances, try 1 completed.
Average CPU Percent: 16.92%
Max CPU Percent: 22.23%
Min CPU Percent: 0.00%
Average Memory Percent: 2.48%
Max Memory Percent: 2.65%
Min Memory Percent: 2.35%

Starting test with 4 concurrent instances, try 1...
Waiting for all challenges to be running...
Test with 4 concurrent instances, try 1 completed.
Average CPU Percent: 23.31%
Max CPU 

Exception: Failed to launch challenge ID 1: {"success":false,"message":"Failed to launch challenge: HTTP 500","error_code":500}